# TVTWebscrape

In [18]:
#modified using extract_text to ensure absence of elements don't cause code to fail
#concat dfs before csv, error in original script

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Function to safely extract text
def extract_text(soup, outer_class, inner_class='field-content'):
    outer_div = soup.find('div', {'class': outer_class})
    if outer_div:
        inner_div = outer_div.find('div', {'class': inner_class})
        return inner_div.text.strip() if inner_div else None
    return None

# Create an empty list to store the dataframes
dfs = []

# Loop through pages
page = 0
while True:
    # Get the page content
    base_url = f'https://tvtpublicreporting.sts.org/tvt-registry?page={page}'
    response = requests.get(base_url)
    bs = BeautifulSoup(response.content, 'html.parser')

    # Regular expression pattern to match '/site/' followed by any number of digits
    pattern = re.compile(r'/site/\d+')

    # Loop through all links with the pattern
    links = bs.find_all('a', href=pattern)

    # If no links are found on the current page, exit the loop
    if not links:
        break


    def safe_extract_text(element):
        return element.text.strip() if element else None

    for link in links:
        url = f'https://tvtpublicreporting.sts.org/{link["href"]}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        print(f'Processing link: {url}')

        # Extract digits from the link
        digits = re.findall(r'\d+', link['href'])
        pid = int(digits[0]) if digits else None

        # Extract hospital and location
        hospital = soup.find('h1').text.strip() if soup.find('h1') else None
        location = extract_text(soup, 'views-field views-field-field-address')

        # Directly extract text for certain fields with a check for None
        composite_score_element = soup.find('div', {'class': 'value'})
        confidence_interval_element = soup.find('div', {'class': 'interval'})
        star_rating_varchar_element = soup.find('div', {'class': 'status'})

        composite_score = safe_extract_text(composite_score_element)
        confidence_interval = safe_extract_text(confidence_interval_element)
        star_rating_varchar = safe_extract_text(star_rating_varchar_element)
 

        procedure_timeframe = extract_text(soup, 'views-field views-field-field-reporting-period')
        cum_vol = extract_text(soup, 'views-field views-field-field-cumulative-tavr-volume')
        first_tavr = extract_text(soup, 'views-field views-field-field-date-of-first-tavr-procedu')
        reporting_vol = extract_text(soup, 'views-field views-field-field-eligible-procedures-in-rep')
        avg_annual_vol = extract_text(soup, 'views-field views-field-field-annual-tavr-volume')

        # Create a dictionary from the scraped information
        data = {'pid': pid, 'hospital': hospital, 'location': location, 'composite_score': composite_score, 'confidence_interval': confidence_interval, 'star_rating_varchar': star_rating_varchar,
                'procedure_timeframe': procedure_timeframe, 'cum_vol': cum_vol, 'first_tavr': first_tavr,
                'reporting_vol': reporting_vol, 'avg_annual_vol': avg_annual_vol}

        # Create a dataframe from the dictionary and append to the list of dataframes
        dfs.append(pd.DataFrame([data]))

    # Increment the page number for the next iteration
    print(f'Finished processing page {page}')
    page += 1

# Concatenate all dataframes in the list
if dfs:
    result = pd.concat(dfs, ignore_index=True)
    # Export the dataframe to a CSV file
    result.to_csv('TVTWebscrape.csv', index=False)
    print("Data exported to TVTWebscrape.csv.")
else:
    print("No data found.")


Processing link: https://tvtpublicreporting.sts.org//site/772319
Processing link: https://tvtpublicreporting.sts.org//site/456326
Processing link: https://tvtpublicreporting.sts.org//site/961195
Processing link: https://tvtpublicreporting.sts.org//site/654912
Processing link: https://tvtpublicreporting.sts.org//site/554000
Processing link: https://tvtpublicreporting.sts.org//site/884031
Processing link: https://tvtpublicreporting.sts.org//site/856498
Processing link: https://tvtpublicreporting.sts.org//site/487702
Processing link: https://tvtpublicreporting.sts.org//site/373631
Processing link: https://tvtpublicreporting.sts.org//site/102309
Processing link: https://tvtpublicreporting.sts.org//site/930946
Processing link: https://tvtpublicreporting.sts.org//site/194521
Processing link: https://tvtpublicreporting.sts.org//site/303521
Processing link: https://tvtpublicreporting.sts.org//site/615332
Processing link: https://tvtpublicreporting.sts.org//site/949340
Processing link: https://